<h1><center>Laboratorio 5: La desperación de Mr. Cheems 🐼</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Primavera 2024</strong></center>

### Cuerpo Docente:

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliar: Eduardo Moya
- Ayudantes: Nicolás Ojeda, Melanie Peña, Valentina Rojas

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Carolina Nuñez
- Nombre de alumno 2: Alonso Uribe


### **Link de repositorio de GitHub:** [Repositorio 💻](https://github.com/carinunez/Labs_MDS/tree/main) 

## Temas a tratar
- Aplicar Pandas para obtener características de un DataFrame.
- Aplicar Pipelines y Column Transformers

## Reglas:

- **Grupos de 2 personas**
- Fecha de entrega: 6 días de plazo con descuento de 1 punto por día. Entregas Jueves a las 23:59.
- Instrucciones del lab el viernes a las 16:15 en formato online. Asistencia no es obligatoria.
- <u>Prohibidas las copias</u>. Cualquier intento de copia será debidamente penalizado con el reglamento de la escuela.
- Tienen que subir el laboratorio a u-cursos y a su repositorio de github. Labs que no estén en u-cursos no serán revisados. Recuerden que el repositorio también tiene nota.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Pueden usar cualquier material del curso que estimen conveniente.

### Objetivos principales del laboratorio
- Comprender cómo aplicar pipelines de Scikit-Learn para generar procesos más limpios en Feature Engineering.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `numpy`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre arreglos (*o tensores*).

## Descripción del laboratorio.

### Importamos librerias utiles 😸

In [17]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
import datetime
from IPython.display import HTML

# Libreria para plotear (En colab esta desactualizado plotly)
# !pip install --upgrade plotly
import plotly.express as px
import plotly.graph_objects as go

# Librerias utiles
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer, StandardScaler

In [18]:
# # Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
# try:
#     from google.colab import drive
#     drive.mount("/content/drive")
#     path = 'Dirección donde tiene los archivos en el Drive'
# except:
#     print('Ignorando conexión drive-colab')

# Feature engineering en datos de retail 🛍️

### 0. Cargar Dataset

<p align="center">
  <img width=300 src="https://s1.eestatic.com/2018/04/14/social/la_jungla_-_social_299733421_73842361_854x640.jpg">
</p>

Mr. Cheems, gerente de una cotizada tienda de retail en Europa, les solicita si pueden analizar los datos de algunas de sus tiendas. En una reunión, Mr Cheems le comenta que la calidad de sus datos no es muy buena, por lo que le solicita a usted que limpie su base de datos y cree nuevos atributos relevantes para el negocio.

Por ello, el área de ventas les entrega archivo llamado `online_retail_data.pickle` el cual usted decide cargar a continuación.

In [ ]:
# Inserte su código aquí
df_retail = pd.read_pickle('online_retail_data.pickle')
print(len(df_retail))
df_retail.head(5)

In [ ]:
df_retail.info()

### 1. Función para explorar características [5 puntos]

<p align="center">
  <img width=300 src="https://editor.analyticsvidhya.com/uploads/47389meme.png">
</p>




Tras inspeccionar brevemente los datos proporcionados, usted decide crear una función que realice lo siguiente:
- Plotee un histograma para las variables precios y cantidad. [3 puntos]
- Imprima un conteo de datos nulos por variable [2 puntos]

NOTA: Para generar los gráficos es **OBLIGATORIO** el uso de plotly.

**Respuesta:**

In [21]:
def explore_data(dataframe_in, *args):
  if len(args)==0 or args is None:
    args = ["Price", "Quantity"]
  
  for column in args:
    fig = px.histogram(dataframe_in, x=column, 
                        title=f'Histograma de {column}',
                        log_y=True,
                       height=500, width=800,
                       )
    fig.update_traces(opacity=0.8)
    fig.show()

  print('Cantidad de valores nulos por variable')
  print(dataframe_in.isna().sum())

In [ ]:
explore_data(df_retail)

### 2. Eliminando outliers [10 puntos]

<p align="center">
  <img width=300 src="https://media.licdn.com/dms/image/C5612AQGdXKCka7HumA/article-cover_image-shrink_600_2000/0/1520056407281?e=2147483647&v=beta&t=VZcfjjzjK4LxXdZkSu1KisWC0Ry8bk4tPCn3R8aYdNM">
</p>




#### 2.1 Creando la clase IQR [5 puntos]

Entre las falencias de los datos, Mr. Cheems le comenta que a veces los operadores no ingresan el precio correcto de los productos. Mr. Cheems le comenta que se dio cuenta de este fenómeno porque hay productos con precios exagerádamente altos o bajos. Por lo cual usted decide eliminar outliers del dataframe a traves del rango intercuartil el cual cuenta con los siguientes pasos:

1. Calcular el primer cuartil $Q1$ y el tercer cuartil $Q3$. Hint: utilice el método `quantile()`

2. Calcular el rango intercuartil (RIC): $RIC = Q3 - Q1$

3. Calcular los límites para identificar outliers:
 - Límite inferior: $~~Q1 - \lambda \cdot RIC$
 - Límite superior: $~~Q3 + \lambda \cdot RIC$

4. Eliminar outliers: Los outliers son los datos que están por debajo del límite inferior o por encima del límite superior.


Para realizar dicha tarea, usted decide crear una clase llamada `IQR()` utilizando `BaseEstimator` y `TransformerMixin` para realizar una transformación de cada una de las columnas numéricas del DataFrame utilizando `ColumnTransformer()` más tarde. Considere que lambda debe ser $\lambda$ un parámetro a definir por el usuario.

Hint: tome como referencia el siguiente [enlace](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer).
>**Nota: No modificar el método set_output de la clase IQR**

**Respuesta:**

In [23]:
class IQR(BaseEstimator, TransformerMixin):

  def __init__(self, param):
    #Completar código aquí
    self.inf = 0
    self.sup = 0
    self.param = param

  def fit(self, X):
    X = pd.DataFrame(X).copy()
    q1 = X.quantile(.25)
    q3 = X.quantile(.75)
    RIC =  q3 - q1
    self.inf = q1 - self.param * RIC
    self.sup = q3 + self.param * RIC
    return self

  def transform(self, X):
    # X = X.loc[(X <= self.sup)&(X >= self.inf)]
    X = pd.DataFrame(X).copy()
    for column, value in self.inf.items():
      X.loc[((X[column] < self.inf.loc[column]) | (X[column] >  self.sup.loc[column])), column] = pd.NA
    return X

  def set_output(self, transform='default'):
    #No modificar esta función
    return self

In [ ]:
iqr = IQR(param=0.4)
df_iqr = iqr.fit(df_retail['Quantity'])
df_iqr.transform(df_retail['Quantity'])

#### 2.2 Creación del Pipeline [5 puntos]

Para comenzar introduciéndose en el uso de pipeline, usted decide definir un pipeline con el Transformer previamente definido. Además, usted decide visualizar cómo cambia la distribución de las variables Precio y Cantidad antes y despues de aplicar IQR. Para ello, usted aplica los siguientes pasos:

- Definir un pipeline llamado `numeric_transformations` para las variables precio y cantidad con la transformación IQR. [1 punto]
- Defina un column transformer que aplique `numeric_transformations` para las variables numéricas y `passthrough` para las variables categóricas. Adicionalmente, fije el parámetro `verbose_feature_names_out` en `False`. Ver hint al final [1 puntos]
- Defina el dataframe `df_iqr` aplicado el column transformer a los datos proporcionados por Mr. Cheems considerando un valor de $\lambda$ que tenga un desempeño aceptable para ambas variables. [1 punto]
- Usar `explore_data` en `df_retail` y en `df_iqr`.  [1 punto]
- Reportar los cambios observados en la distribución de las variables. ¿Qué sucede al aumentar el valor de lambda? [1 punto]


HINT: El transformador `passthrough` está predefinido y es una opción que puedes usar para las columnas que no deseas transformar. Al especificar 'passthrough' para una parte de tu ColumnTransformer, las columnas correspondientes pasarán a través del ColumnTransformer sin ninguna modificación. El siguiente [enlace](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) le puede ser útil.
>**Nota: Mantenga el método set_output del column transformer con la transformación `pandas` para obtener un dataframe una vez aplicado el column transformer.**

**Respuesta:**

Apóyese de la siguiente estructura para su respuesta:

In [ ]:
# Definicion las variables que pasarán por cada pipeline
numerical_columns = (df_retail.select_dtypes('float').columns.to_list())
print(numerical_columns)
categorical_columns = (df_retail.select_dtypes(['category', 'object', 'datetime64[ns]']).columns.to_list())
print(categorical_columns)
# 1. Definicion del pipeline
numeric_transformations = Pipeline(steps=[('IQR Transformation', IQR(param=0.3))])

# 2. ColumnTransformer
column_transformer = ColumnTransformer(
                                    [('numerical', numeric_transformations, numerical_columns),
                                        ('categorical', 'passthrough', categorical_columns)],
                                        verbose_feature_names_out=False)

column_transformer.set_output(transform='pandas')

# 3. Aplicamos ColumnTransformer a los datos
df_iqr = column_transformer.fit_transform(df_retail)

# 4. Gráficos 
print('df_retail:')
explore_data(df_retail)
print()
print('df_iqr:')
explore_data(df_iqr)

5. Se nota que las distribuciones cambiaron. Se eliminaron los valores extremos, en el caso de Price se eliminaron los valores cercanos a cero, siendo bastante $~10^6$. Para Quantity enseguida se pierden cantidades mayores a 20, toda las cantidades compradas rondan esta cifra.

In [ ]:
# Cambios en la distribución de las variables -> Gridsearch
for p in [0.1, 0.8, 0.5, 0.3]:
    print()
    print(f"Probando transformación con parámetro lambda: {p}")
    numeric_transformations = Pipeline(steps=[('IQR Transformation', IQR(param=p))])
    column_transformer = ColumnTransformer(
                                        [('numerical', numeric_transformations, numerical_columns),
                                            ('categorical', 'passthrough', categorical_columns)],
                                            verbose_feature_names_out=False)
    column_transformer.set_output(transform='pandas')
    df_iqr = column_transformer.fit_transform(df_retail)

    print(f'df_iqr_{p}:')
    explore_data(df_iqr)

5. Mientras más grande el parámetro $\lambda$ más valores extremos son considerados.

*Reporte los cambios observados aquí.*

### 3. Agregando un imputer al pipeline [10 puntos]



<p align="center">
  <img width=300 src="https://media.makeameme.org/created/hmm-there-is.jpg">
</p>

Para continuar con la limpieza del dataframe usted decide imputar los datos nulos de las variables numéricas, para lo cual decide realizar las siguientes tareas:

1. Crear un pipeline para variables categóricas llamado `categoric_transformations` con un paso llamado `mode_imputer`, en el cual se imputen los datos faltantes por la categoría más frecuente.
2. Agregar al pipeline `numeric_transformations` un paso llamado `mean_imputer`, en el cual se imputen los datos por la media usando [SimpleImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) [1 punto]
3. Crear y aplicar un `ColumnTransformer` actualizado con los pipelines `categoric_transformations` y `numeric_transformations` a `df_retail`, creando un dataframe llamado `df_mean_imputer`. [1 punto]
4. Comparar los resultados de `explore_data` en `df_mean_imputer` y `df_iqr`. ¿Qué diferencias observa en la distribución de los datos? [2 puntos]
5. Cambiar el imputer de `numeric_transformations` por [KNNImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html) y definir un nuevo dataframe llamado `df_knn_imputer`, aplicando el nuevo ColumnTransformer a `df_retail`. En caso de los tiempos de ejecución sean altos puede probar a reducir el parámetro `n_neighbors`. [1 punto]
6. Comparar los resultados de `explore_data` en `df_knn_imputer` y `df_iqr`. ¿Qué diferencias observa en la distribución de los datos? [2 puntos]
7. Comparar los resultados de `explore_data` en `df_knn_imputer` y `df_mean_imputer`. ¿Cuál método de imputación es mejor? Deje el método escogido en el ColumnTransformer. [2 puntos]

>**Nota: Fije el parámetro verbose_feature_names_out en `False` y utilice el método set_output con transformación `pandas` en cada ColumnTransformer para obtener como salida un dataframe.**

**Respuesta:**

In [ ]:
from sklearn.impute import SimpleImputerS

# 1. imputar categoria más frecuente
categoric_transformations = Pipeline([('mode_imputer', SimpleImputer(strategy='most_frequent'))])

# 2. imputar media 
numeric_transformations = Pipeline([('mean_imputer', SimpleImputer(strategy='mean'))])

# 3. Column Tranformer
new_col_transform = ColumnTransformer([
            ('categoric_transformations', categoric_transformations, categorical_columns),
            ('numeric transformations', numeric_transformations, numerical_columns)],
            verbose_feature_names_out=False)

new_col_transform.set_output(transform='pandas')

df_mean_imputer = new_col_transform.fit_transform(df_retail)

# 4. Comparar data df_iqr y df_mean_imputer
print('df_iqr: ')
explore_data(df_iqr)
print('df_mean_imputer: ')
explore_data(df_mean_imputer)

ya no hay valores nulos para mean_imputer, pero permanecen los outliers. Las variables categoticas se seleccionan por más frecuentes.

In [48]:
# 5. Usar KNNImputer en numeric_transformations
from sklearn.impute import KNNImputer

numeric_transformations = Pipeline([('knn_imputer', KNNImputer(n_neighbors=2, weights='distance'))])
knn_transform = ColumnTransformer([
                        ('numeric transformations', numeric_transformations, numerical_columns)],
                        verbose_feature_names_out=False)

knn_transform.set_output(transform='pandas')

df_knn = knn_transform.fit_transform(df_retail)
df_knn_imputer = pd.DataFrame(df_knn, columns=numerical_columns)


In [ ]:

# 6. Comparar data df_knn_imputer y df_iqr
print('df_iqr: ')
explore_data(df_iqr)
print('df_knn_imputer: ')
explore_data(df_knn_imputer)

6. 7. Lo más notorio de la transformación es su costo. Para dos vecinos el costo es demasiado y no se observa mayor diferencia. Va a depender del uso que se le de a los datos, pero iqr sigue siendo el mejor hasta ahora, mean imputer podría añadir mucho sesgo a la muestra dependiendo de la cantidad de nulls, que para el caso de la variable Quantity son bastantes. 

*Escriba su respuesta aquí*

### 4. Creación de nuevas features [20 puntos]

<p align="center">
  <img width=250 src="https://miro.medium.com/max/1000/1*JtTWgAcfVTWV8OTjT47Atg.jpeg">
</p>


#### 4.1 Definicion de LRMFP [10 puntos]
(2 puntos por cada custom feature)

Dado que Mr. Lepin está interesado en obtener nuevos atributos relevantes para su negocio, su equipo de expertos sugiere la construcción de variables **LRMFP**, las que se construyen en base a las siguientes definiciones:

- **Length (L)**: Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.

- **Recency (R)**: Indica hace cuanto tiempo el cliente realizo su ultima compra. Notar que para este caso, mientras mas grande es el valor, menos interes posee el usuario para repetir una compra en uno de los locales. **Considere "hoy" como la fecha mas reciente del dataset**.

- **Monetary (M)**: El término "monetario" se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.

- **Frequency (F)**: Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente.

- **Periodicity (P)**: Representa si los clientes visitan las tiendas con regularidad.

$$Periodicity(n)=std(IVT_1, ..., IVT_n)$$

Donde $IVT$ denota el tiempo entre visitas y n representa el número de valores de tiempo entre visitas de un cliente.


$$IVT_i=date\_diff(t_{i+1},t)$$

En base a las definiciones señaladas, diseñe una función que permita obtener las características **LRMFP** recibiendo un DataFrame como entrada. Para esto, no estará permitido el uso de iteradores, utilice todas las herramientas que les ofrece `pandas` para realizar esto.

Una referencia que le puede ser útil es el [documento original](https://www.researchgate.net/publication/315979555_LRFMP_model_for_customer_segmentation_in_the_grocery_retail_industry_a_case_study) en donde se propone este método.

**<u>Formato</u> del Resultado Esperado:**

| Customer ID | Length | Recency | Frequency | Monetary | Periodicity |
|------------:|-------:|--------:|----------:|---------:|------------:|
|   12346.0   |    294 |      67 |        46 |   -64.68 |        37.0 |
|   12347.0   |     37 |       3 |        71 |  1323.32 |         0.0 |
|   12349.0   |    327 |      43 |       107 |  2646.99 |        78.0 |
|   12352.0   |     16 |      11 |        18 |   343.80 |         0.0 |
|   12356.0   |     44 |      16 |        84 |  3562.25 |        12.0 |

**Respuesta:**

In [ ]:
df_retail.head(2)

In [51]:
def custom_features(dataframe_in: pd.DataFrame):
    dataframe = dataframe_in.copy()
    
    # Length
    _min = dataframe.loc[:,["Customer ID", "InvoiceDate"]].groupby("Customer ID").min()
    _max = dataframe.loc[:,["Customer ID", "InvoiceDate"]].groupby("Customer ID").max()
    length = _max - _min
    length = length.applymap(lambda deltatime: deltatime.days).rename(columns={"InvoiceDate":"Length"})
    
    # Recency
    recency = pd.Timestamp.today() - _max
    recency = recency.applymap(lambda deltatime: deltatime.days).rename(columns={"InvoiceDate":"Recency"})

    # Monetary
     # Asumimos una fecha en especifico como una sola visita, sumando cada compra, para luego tomar la media de todas las visitas
    df_period = dataframe.loc[:,["Customer ID", "InvoiceDate", "Price", "Quantity"]]
    df_period["TotalPrice"] = df_period.loc[:,["Price", "Quantity"]].aggregate(lambda row: row[0]*row[1], axis=1)
    df_period["PeriodDate"] = df_period["InvoiceDate"].apply(lambda date: date.date())
    df_period.drop(labels=["InvoiceDate", "Quantity", "Price"], axis=1, inplace=True)
    sum_expenses_per_date = df_period.groupby(["Customer ID", "PeriodDate"]).sum()
    sum_expenses_per_date["TotalPrice"] = sum_expenses_per_date["TotalPrice"].apply(lambda price: pd.NA if price==0 else price)
    sum_expenses_per_date.dropna(inplace=True)
    monetary = sum_expenses_per_date.groupby("Customer ID").mean().rename(columns={"TotalPrice":"Monetary"})

    # Frecuency
     # Nuevamente se considera una visita como una fecha en particular
    frecuency = sum_expenses_per_date.groupby("Customer ID").count().rename(columns={"TotalPrice":"Frecuency"})

    # Periodicity
    def ivt(serie:pd.Series):
        # Recibe un dataframe ordenado por CustomID y Fechas de Visita.
        date = serie.copy().reset_index()
        date_i = date.copy()
        date_i.index += 1
        delta = (date["PeriodDate"] - date_i["PeriodDate"]).apply(lambda deltatime: deltatime.days)
        delta = delta.mask(delta.isnull(), other=np.nan).dropna()
        return delta.std() if len(delta) > 0 else pd.NA
    dates = sum_expenses_per_date.copy().reset_index("PeriodDate").drop(columns="TotalPrice").sort_values(["Customer ID", "PeriodDate"], axis=0)
    dates = dates.reset_index("Customer ID").reset_index(drop=True)
    dates["PeriodDate"] = pd.to_datetime(dates["PeriodDate"], format="%Y-%m-%d")
    periodicity = dates.groupby("Customer ID").aggregate(ivt).rename(columns={"PeriodDate":"Periodicity"})

    return pd.concat([length, recency, monetary, frecuency, periodicity], axis=1).reset_index("Customer ID")
    


In [ ]:
custom_features(df_retail)

#### 4.2 Agregando las custom features [10 puntos]

Ahora, usted decide agregar al pipeline las nuevas variables creadas, para lo cual realiza las siguientes tareas:

1. Cree un nuevo pipeline llamado `retail_pipeline` que encapsule el ColumnTransformer y calcule las LRMFP. El primer paso del pipeline llámelo  `col_tranformer` y el segundo paso llámelo `custom_features`, incorpora las nuevas variables al dataframe. Hint: les puede ser útil investigar [este](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.FunctionTransformer.html) método. [1 punto]
2. Aplicar el pipeline actualizado a los datos proporcionados por Mr. Cheems, creando un nuevo dataframe llamado `df_custom`. [1 punto]
3. Explorar la distribución de las nuevas variables con `explore_data` y comentar brevemente (2-3 líneas) características de cada custom feature. [5 puntos]
5. Entregar un insight para el negocio en base a las nuevas variables. [3 puntos]

>Nota: Recuerde fijar el parámetro `verbose_feature_names_out` en `False` e incorporar el método `set_output` para obtener una salida en formato dataframe del ColumnTransformer.

**Respuesta**

In [42]:
from sklearn.preprocessing import FunctionTransformer
# 1.
retail_pipeline = Pipeline([('col_transformer', column_transformer), ('custom_features', FunctionTransformer(func=custom_features))])

In [ ]:
# 2. 
df_custom = retail_pipeline.fit_transform(df_retail)
df_custom = df_custom.convert_dtypes('float64')

In [ ]:
explore_data(df_custom, "Length", "Recency", "Monetary", "Frecuency", "Periodicity")

3. Explorar la distribución de las nuevas variables con `explore_data` y comentar brevemente (2-3 líneas) características de cada custom feature. [5 puntos]

**Length:** La mayoria de los clientes son recientes, hay un grupo que son clientes habituales durante un largo periodo

**Recency:** La mayoria de los clientes han vuelvo "ultimamente". Hay una ausencia de clientes en una fecha en particular. La varianza de la distribución resulta considerable y poco concluyente.

**Monetary:** La moda de gastos medios para los clientes es de al rededor de 110-120. Existen 6 clientes en particular, que gastan al rededor de 700 dolares por visita. La matoría de clientes ronda entre los 0 y 500 dolares medios gastados

**Frecuency:** En su mayoria y con poca varianza los clientes no frecuentan el local. Existen ciertos outliers o clientes predilectos q lo frecuentan "siempre".

**Periodicity:** Para los clientes q sí vuelven a la tienda. El tiempo moda transcurrido entre visitas, es de alrededor de 17 a 20 días, lo que hace sentido si uno considera el reabastecimiento de los productos ofrecidos por la tienda. La varianza de la distribución, en cualquier caso, es considerable.

5. Entregar un insight para el negocio en base a las nuevas variables. [3 puntos]
La varianza amplía de Periodicity podría implicar que los clientes no son fieles a la tienda, posiblement retornan por un producto único en el local, en particular. Del mismo modo nos lo indica Recency; los cleintes vuelven, en su mayoría "no frecuentemente". Esto podría indicarnos que hay un problema de fidelidad de clientes. Luego de cofirmar esto, se podrían especular las causas, por ejemplo: altos precios, distancia de viaje a la tienda, ordenamiento de productos, marketing, etc.  

### 5. MinMax Scaler [10 puntos]

<p align="center">
  <img width=300 src="https://i.imgflip.com/1fsprn.jpg">
</p>


#### 5.1 Definición del Column Transformer [5 puntos]

Construya una clase llamada `MinMax()` para realizar una transformación de cada una de las columnas de un DataFrame utilizando `ColumnTransformer()`. Recuerde  usar `BaseEstimator` y `TransformerMixin`.


 Para esto considere que Min-Max escaler queda dada por la ecuación:

$$MinMax = \dfrac{x-min(x)}{max(x) - min(x)}$$


Consulte el siguiente [link](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer) si tiene dudas sobre la creación de custom transformers.

**Respuesta:**

In [53]:
class MinMax(BaseEstimator, TransformerMixin):

    def fit(self,X):
        #Agregar código aquí
        self.min = np.min(X, axis=0)
        self.max = np.max(X,axis=0)
        return self

    def transform(self,X):
        #Agregar código aquí
        X = (X - self.min)/(self.max - self.min)
        return X

    def set_output(self,transform='default'):
        #No modificar este método
        return self

In [ ]:
mm = MinMax()
mm.fit(df_retail.Quantity).transform(df_retail.Quantity)

#### 5.2 Incorporando MinMax al pipeline [5 puntos]

Ahora, usted decide agregar el escalamiento al pipeline, para lo que decide seguir los siguientes pasos:

- Agregar el paso `minmax` al pipeline `numeric_transformations`, haciendo uso de la clase creada. [1 punto]
- Defina el dataframe `df_minmax` aplicando el ColumnTransformer actualizado a los datos proporcionados por Mr. Cheems. [1 punto]
- Usar `explore_data` en `df_retail` y en `df_minmax`. [1 punto]
- Reportar los cambios observados en la distribución de las variables.  [2 puntos]

>Nota: Recuerde fijar el parámetro `verbose_feature_names_out` en `False` e incorporar el método `set_output` para obtener una salida en formato dataframe del ColumnTransformer.

**Respuesta:**

Se decide, 

In [ ]:
numeric_transformations = Pipeline([('IQR Transformation', IQR(param=0.3)),
                                    ('minmax', MinMax())])
iqr_minmax_transform = ColumnTransformer([
                        ('numeric transformations', numeric_transformations, numerical_columns)],
                        verbose_feature_names_out=False)

iqr_minmax_transform.set_output(transform='pandas')

df_minmax = iqr_minmax_transform.fit_transform(df_retail)

print('df_retail')
explore_data(df_retail)
print('df_minmax')
explore_data(df_minmax)


Se eliminan los outliear y luego se aplica minmax. Las distribuciónes son parecidas, pero los límites estan definido. por lo tanto valores similares quedan más compactos, mientras valores extremos definen los límites. 

### 6. Pregunta teórica [5 puntos]

<p align="center">
  <img width=300 src="https://file.coinexstatic.com/2023-09-19/166BAC031F222E5910954E7D7D0BC844.png">
</p>

Finalmente, explíquele a Mr. Cheems porqué es útil la creación de pipelines al momento de hacer Feature Engineering en Machine Learning.

**Respuesta:**

Las Pipelines al ser listas de tuplas que contienen todos los pasos que se deben realizar a los datos, permiten reutilizar código, ajustar hiperparámetros fácilmente, procesar los datos de manera consistente, entre otros.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por el foro de U-cursos o por correo.

![Gracias Totales!](https://i.pinimg.com/originals/65/ae/27/65ae270df87c3c4adcea997e48f60852.gif "bruno")


<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>